In [2]:
import warnings
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import datetime
import itertools
from sqlalchemy import create_engine
sys.path.append("C:/dev/pycmqlib3/")
sys.path.append("C:/dev/pycmqlib3/misc_scripts/")
warnings.filterwarnings('ignore')
import misc
import data_handler as dh
import backtest
import dbaccess
cnx = dbaccess.connect(**dbaccess.dbconfig)

stmt = "select * from hist_fut_daily where exch = 'CZCE' and date<'2015-01-01'"
df = pd.io.sql.read_sql(stmt, cnx)
print(len(df))

99876


In [3]:
df['symbol'] = df['instID']
for i in range(5, 15):
    flag = (df['instID'].str[2] == str(i % 10)) & (df['date']<=datetime.date(2000+i, 12, 31))
    df.loc[flag, 'symbol'] = df.loc[flag, 'instID'].apply(lambda x: ''.join([x[:2], str(i//10), x[2:]]))

df['instID'] = df['symbol']
df.drop(columns = ['symbol'], inplace = True)
print(len(df), df.head(30))

99876     instID  exch        date     open     high      low    close   settle  \
0   CF1001  CZCE  2009-01-21  13000.0  13000.0  13000.0  13000.0  13000.0   
1   CF1001  CZCE  2009-02-04  12715.0  13000.0  12715.0  12920.0  12910.0   
2   CF1001  CZCE  2009-02-06  12950.0  12975.0  12905.0  12910.0  12935.0   
3   CF1001  CZCE  2009-02-09  13000.0  13100.0  12975.0  13055.0  13025.0   
4   CF1001  CZCE  2009-02-10  13090.0  13090.0  13090.0  13090.0  13090.0   
5   CF1001  CZCE  2009-02-11  13040.0  13040.0  13035.0  13035.0  13040.0   
6   CF1001  CZCE  2009-02-12  13000.0  13000.0  13000.0  13000.0  13000.0   
7   CF1001  CZCE  2009-02-13  13100.0  13100.0  13000.0  13030.0  13020.0   
8   CF1001  CZCE  2009-02-16  13025.0  13030.0  13025.0  13030.0  13025.0   
9   CF1001  CZCE  2009-02-17  13050.0  13070.0  13025.0  13045.0  13040.0   
10  CF1001  CZCE  2009-02-18  13030.0  13035.0  12940.0  13000.0  13010.0   
11  CF1001  CZCE  2009-02-19  13020.0  13025.0  12800.0  12830.0  1291

In [4]:
cursor = cnx.cursor()
stmt = "delete from hist_fut_daily where exch = 'CZCE' and date<'2015-01-01'"
cursor.execute(stmt)
#cnx.commit()

In [5]:
conn = create_engine('mysql+mysqlconnector://{user}:{passwd}@{host}/{dbase}'.format( \
                    user = dbaccess.dbconfig['user'], \
                    passwd = dbaccess.dbconfig['password'],\
                    host = dbaccess.dbconfig['host'],\
                    dbase = dbaccess.dbconfig['database']), echo=False)


In [6]:
df.to_sql('hist_fut_daily', con = conn, if_exists='append', index=False, method = dbaccess.mysql_replace_into)


In [10]:
stmt = "select * from hist_fut_daily where exch = 'DCE' and char_length(instID)=4"
df = pd.io.sql.read_sql(stmt, cnx)
print(len(df))


25302


In [12]:
df['instID'] = df['instID'].apply(lambda x: ''.join([x[:1], '0', x[1:]]))
print(df.head(20))

   instID exch        date    open    high     low   close  settle  volume  \
0   a0701  DCE  2007-01-04  2672.0  2673.0  2606.0  2606.0  2660.0     669   
1   a0701  DCE  2007-01-05  2660.0  2674.0  2615.0  2660.0  2651.0    1510   
2   a0701  DCE  2007-01-08  2662.0  2670.0  2661.0  2665.0  2663.0     541   
3   a0701  DCE  2007-01-09  2668.0  2668.0  2651.0  2651.0  2657.0      64   
4   a0701  DCE  2007-01-10  2645.0  2675.0  2640.0  2675.0  2655.0     475   
5   a0701  DCE  2007-01-11  2670.0  2700.0  2670.0  2680.0  2687.0     321   
6   a0701  DCE  2007-01-12  2690.0  2690.0  2666.0  2675.0  2675.0     360   
7   a0701  DCE  2007-01-15  2730.0  2780.0  2727.0  2780.0  2750.0     348   
8   a0701  DCE  2007-01-16  2640.0  2710.0  2640.0  2699.0  2698.0    1319   
9   a0701  DCE  2007-01-17  2660.0  2730.0  2660.0  2730.0  2676.0     323   
10  a0703  DCE  2007-01-04  2746.0  2746.0  2695.0  2696.0  2713.0      55   
11  a0703  DCE  2007-01-05  2704.0  2704.0  2704.0  2704.0  2704

In [13]:
cursor = cnx.cursor()
stmt = "delete from hist_fut_daily where exch = 'DCE' and char_length(instID)=4"
cursor.execute(stmt)
#cnx.commit()

In [14]:
df.to_sql('hist_fut_daily', con = conn, if_exists='append', index=False, method = dbaccess.mysql_replace_into)